In [ ]:
import numpy as np
import pandas as pd
import os

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')
data.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

data['CPM'] = data.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)
data = data[(data['CPM'] >= 0) & (data['CPM'] < data['CPM'].quantile(0.95))]
# Удаляем total_revenue и measurable_impressions, как прямо используемые в рассчете таргета
data.drop(columns=['total_revenue', 'measurable_impressions'], inplace=True)

In [ ]:
train = data[data['date'] < '2019-06-18']
val = data[(data['date'] >= '2019-06-18')&(data['date'] < '2019-06-22')]

X_train = train.drop(columns=["date","CPM"])
y_train = train['CPM']
X_val = val.drop(columns=["date","CPM"])
y_val = val['CPM']

lgb = LGBMRegressor(n_estimators=2500)
lgb.fit(X_train, y_train)
y_pred = lgb.predict(X_val)

print(f"My awesome and not overfitted model scored {mean_squared_error(y_val, y_pred):.2f} on val set.")

In [ ]:
train = data[data['date'] < '2019-06-22']
test = data[data['date'] >= '2019-06-22']

X_train = train.drop(columns=["date","CPM"])
y_train = train['CPM']
X_test = test.drop(columns=["date","CPM"])
y_test = test['CPM']

lgb = LGBMRegressor(n_estimators=2500)
lgb.fit(X_train, y_train)
y_pred = lgb.predict(X_test)

print(f"My awesome and not overfitted model scored {mean_squared_error(y_test, y_pred):.2f} on test set.")